In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 9 데이터 수집과 그룹 연산
- key를 이용하여 객체를 여러조각으로 나누기
- 요약통계
- column에 함수 적용
- 피벗 테이블, 교차 일람표
- 변위치 분석,

## 9.1 Groupby 메카닉
분리 - 적용 - 결합

group의 기준 : 주로 범주형 데이터(categorical data)

In [2]:
df = pd.DataFrame({ 'key1' : list('aabba'),
                  'key2' : list('one,two,one,two,one'.split(',')),
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


#### groupby 객체 : 연산을 수행하기 전 준비상태(setting)

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.354938
b   -0.585474
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
# 계층적 색인인 Series를 반환한다.

key1  key2
a     one     0.271976
      two     0.520862
b     one     0.178584
      two    -1.349532
Name: data1, dtype: float64

In [6]:
means.unstack() # 따라서 Series완 관련된 메서드 수행이 가능

key2,one,two
key1,,
a,0.271976,0.520862
b,0.178584,-1.349532


In [8]:
states = np.array(['ohio','califor','califor','ohio','ohio'])
years = np.array([2005,2005,2006,2005,2006])

dataframe 안에 존재하지 않는 group key로 groupby 연산

In [9]:
df['data1'].groupby([states, years]).mean()
# DataFrame 안에 존재하지 않는 외부 group key 로도 그룹화 가능(같은 길이를 같는 group key만)

califor  2005    0.520862
         2006    0.178584
ohio     2005    0.068185
         2006   -0.941951
Name: data1, dtype: float64

#### key 는 해당 데이터프레임 안에서 찾는 경우가 많다

groupby( ) 메서드 인자로 인덱스만 써줘도 상관없다(가공하려는 데이터프레임 안의 key를 사용하는 경우)

연산의 결과 값은 type 이 numerical 인 열이나 행만 보여준다. (여기서 key2는 범주형이므로 결과값에서 제외)

In [10]:
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


In [11]:
# dataframe 안에 key2 column이 있지만 numeric type 이 아니므로 연산에서 제외
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.354938,0.395297
b,-0.585474,0.095763


In [20]:
df

,key1,key2,data1,data2
0,a,one,0.342451,-0.140495
1,a,two,0.212360,0.680066
2,b,one,1.519154,1.506582
3,b,two,0.871806,-0.139929
4,a,one,0.235065,-0.475405


In [14]:
# 범주형 데이터의 경우 각 경우를 count 하는 일이 많다.
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 9.1.1 그룹 간 순회하기

iteration 지원 : 튜플로 반환 ( 그룹이름,  매칭되는 데이터 )

groupby 객체: [그룹 key 에 해당하는 이름], [그에 매칭되는 데이터] 로 나눌 수 있다.

In [15]:
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


In [16]:
for name,group in df.groupby('key1'): # key1 의 내부의 데이터(범주형, 여기서는 a와 b)에 따라 '이름'과 '데이터 프레임' 반환
    print(name)
    print('-'*35)
    print(group)
    print('='*35)
    
# groupby 메서드를 적용한 groupby 객체는 (group의 키 , 그에 속한 데이터들) 쌍으로 이루어진다.
# 아래의 예제


a
-----------------------------------
  key1 key2     data1     data2
0    a  one  1.485902  1.300076
1    a  two  0.520862 -0.482945
4    a  one -0.941951  0.368759
b
-----------------------------------
  key1 key2     data1     data2
2    b  one  0.178584  0.277137
3    b  two -1.349532 -0.085610


In [29]:
for (k1,k2), group_data in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group_data)
    
# group key 가 여러개인 경우에는 그에 맞게 튜플로 묶어서 표현해준다.

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.342451 -0.140495
4    a  one  0.235065 -0.475405
('a', 'two')
  key1 key2    data1     data2
1    a  two  0.21236  0.680066
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.519154  1.506582
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.871806 -0.139929


In [17]:
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


In [18]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

### data type 별로 그룹화하기
    dataframe객체.groupby( df.dtype ) : group key 를 data type 으로 준다.

In [35]:
grouped2 = df.groupby(df.dtypes , axis = 1) 
#data frame 은 열마다 하나의 데이터타입만 저장하므로, 같은 열끼리 그룹화하려는 시도에서는 
# axis = 1(열) 로 주어야한다.
grouped2

# datatype 을 인자로 주어 datatype 별로 그룹화할 수 있다.

In [36]:
for (groupname, data) in grouped2:
    print(groupname)
    print('-'*30)
    print(data)
    print('='*30)
    
# 데이터 타입이 float64  인 그룹 :'data1' 'data2' 열과 해당 데이터
# 데이터 타입이 object 인 그룹 : 'key1' 'key2' 열과 해당 데이터

float64
------------------------------
      data1     data2
0  0.342451 -0.140495
1  0.212360  0.680066
2  1.519154  1.506582
3  0.871806 -0.139929
4  0.235065 -0.475405
object
------------------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [39]:
dict(list(grouped2))

{dtype('float64'):       data1     data2
 0  0.342451 -0.140495
 1  0.212360  0.680066
 2  1.519154  1.506582
 3  0.871806 -0.139929
 4  0.235065 -0.475405, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

## 9.1.2 칼럼 또는 칼럼의 일부만 선택하기
groupby 객체를 column 이름이 담긴 배열로 색인
    
    dataframe객체.groupby( 키값들 )[인자] 
- 인자가 리스트나 배열 : DataFrameGroupby 객체 생성
- 인자가 단일 값 : SeriesGroupby 객체 생성

In [42]:
df

,key1,key2,data1,data2
0,a,one,0.342451,-0.140495
1,a,two,0.212360,0.680066
2,b,one,1.519154,1.506582
3,b,two,0.871806,-0.139929
4,a,one,0.235065,-0.475405


In [46]:
df['data1'].groupby(df['key1'], axis = 0).mean() # data1 의 열에 대해 key1 열의 데이터를 기준으로 그룹화 한 후 mean 값 계산

key1
a    0.263292
b    1.195480
Name: data1, dtype: float64

## 9.1.3 사전과 Series에서 묶기
#### 그룹정보가 배열 형태가 아닌 경우

In [19]:
people = pd.DataFrame(np.random.randint(20, size = 5*5).reshape(5,5), 
                      columns = list('abcde'),
                      index = 'joe,steve,wes,jim,travis'.split(','))
people

,a,b,c,d,e
joe,0,19,1,1,7
steve,7,15,16,7,9
wes,19,1,10,17,18
jim,8,19,10,10,5
travis,11,6,10,8,3


In [20]:
people.ix[2:3,['b','c']] = np.nan# 일부값을 nan 으로 바꿔주자
people # 

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,a,b,c,d,e
joe,0,19.0,1.0,1,7
steve,7,15.0,16.0,7,9
wes,19,NaN,NaN,17,18
jim,8,19.0,10.0,10,5
travis,11,6.0,10.0,8,3


In [21]:
mapping = dict(zip(list('abcdef'), ['red','red','blue','blue','red','orange']))
mapping

# a,b,e - red
# c,d - blue
# f - orange 에 매칭되어 그룹화 된다. f 는 없기 때문에 무시된다.

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

#### groupby( 딕셔너리 )

In [23]:
# axis = 1로 했으므로 열에 대해 묶는다.
# groupby(딕셔너리) : 내부 딕셔너리는 {열이름 : 범주, ....} 로 되어 있다.
# 따라서, 범주끼리 묶여 연산
by_col = people.groupby(mapping, axis = 1) # axis = 1 : 열에 따라 그룹화를 실행 , 즉 가로방향으로
by_col.sum()

,blue,red
joe,2.0,26.0
steve,23.0,31.0
wes,17.0,37.0
jim,20.0,32.0
travis,18.0,20.0


#### groupby( series 형 )

In [24]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [25]:
# series의 인덱스(열 이름) 와 값(범주)
# 범주끼리 묶여 연산 수행
people.groupby(map_series, axis = 1).count()

,blue,red
joe,2,3
steve,2,3
wes,1,2
jim,2,3
travis,2,3


## 9.1.4 함수로 묶기
    groupby( 함수 ) : 인자 부분에 함수 반환 값을 넣어 그룹화

파이썬 함수를 이용하여 그룹을 매핑하는 것 : 독창적 , 추상적
 
즉, groupby ( 함수 ) 를 사용 : key 값에 함수를 넣어 그룹으로 매핑

In [50]:
people

,a,b,c,d,e
joe,7,3.0,14.0,12,2
steve,15,17.0,14.0,13,5
wes,19,NaN,NaN,3,11
jim,2,6.0,7.0,0,6
travis,3,9.0,13.0,2,1


In [25]:
people.groupby(len).sum() # index ( joe, steve, wes ...) 들의 문자열 길이를 기준으로 그룹핑한 것
# 3 : joe, wes, jim
# 5 : steve
# 6 : travis

,a,b,c,d,e
3,27,38.0,11.0,28,30
5,7,15.0,16.0,7,9
6,11,6.0,10.0,8,3


In [60]:
# 열에 대해 수행하면 a,b,..,e 모두 len 값이 1 이므로 다음과 같이 묶여서 나온다.
people.groupby(len, axis=1).sum()

,1
joe,38.0
steve,64.0
wes,33.0
jim,21.0
travis,28.0


In [63]:
people

,a,b,c,d,e
joe,7,3.0,14.0,12,2
steve,15,17.0,14.0,13,5
wes,19,NaN,NaN,3,11
jim,2,6.0,7.0,0,6
travis,3,9.0,13.0,2,1


In [29]:
np.sign(people.isnull().sum())

a    0
b    1
c    1
d    0
e    0
dtype: int64

In [32]:
people.groupby(np.sign(people.isnull().sum()), axis = 1).sum()

,0,1
joe,8.0,20.0
steve,23.0,31.0
wes,54.0,0.0
jim,23.0,29.0
travis,22.0,16.0


### 9.1.5 색인 단계로 묶기

multiIndex 인 index 나 column에서 groupby 기준을 찾아서 연산

    dataframe객체.groupby(계층적 색인)
            level 인자를 통해 처리

In [33]:
col = pd.MultiIndex.from_arrays([['us','us','us','jp','jp'],[1,3,5,1,3]], names = ['cty','tenor'])
col

MultiIndex(levels=[['jp', 'us'], [1, 3, 5]],
           codes=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [34]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = col)
hier_df

cty          us                            jp          
tenor         1         3         5         1         3
0      0.165537 -1.645904  0.857854  0.621726 -1.028421
1     -0.311212 -1.294681  0.207183  0.132950 -0.144177
2      0.390921 -0.160669 -0.042591  0.747489  0.369609
3     -1.323710 -0.105175  1.361504 -0.047087  0.888184

In [30]:
hier_df.groupby(level = 'cty', axis = 1).mean() # 열이 계층적 색인이므로 level 을 명시하여야 한다.

cty,jp,us
0,-0.971856,-0.570706
1,-0.186825,0.525747
2,-0.725124,-0.788975
3,-0.184656,0.666910


## 9.2 데이터 수집

배열 -> 스칼라(mean, sum, count, min, max)
    
    dataframe객체.groupby( ).사용하고 싶은 함수

#### 사용자가 정의한 함수 사용
    groupby().agg( 정의한 함수 ) : agg 메서드는 df에 함수를 적용하게 해 준다.

#### 최적화 된 groupby 메서드
    count : na 값이 아닌 수
    sum
    mean
    median : 산술 중간 값
    std, var 
    min, max
    prod : 값의 곱
    first ,last : 첫번째 값과 마지막 값

In [36]:
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


In [37]:
df.groupby('key1').quantile(0.9) # group으로 묶은 후 각각에 대해 qunatile을 구했다.

# a를 가진 데이터를 한 그룹으로 묶었을 때 그 데이터들 중 quantile 0.9 인 값
# b를 가진 데이터를 한 그룹으로 묶었을 때 그 데이터들 중 quantile 0.9 인 값

0.9,data1,data2
key1,,
a,1.292894,1.113812
b,0.025773,0.240862


In [39]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [40]:
#grouped = df['data1'].groupby(df['key1'])
grouped.agg(peak_to_peak) # grouped : data1 열을 key1으로 그룹화 한 groupby 객체

key1
a    2.427853
b    1.528117
Name: data1, dtype: float64

## 9.2.1 칼럼에 여러 가지 함수 적용하기
#### 모든 칼럼에 여러 함수 적용  
    dataframe객체.agg( [ func1, func2, ....] )
    dataframe객체.agg( [ (결과 반환 열 이름 , func1) , (  ,  ) ....] ) 
#### 칼럼마다 다른 함수 적용 / 딕셔너리로 {'열' : 적용함수 ...} 로 agg 에 인자로 준다. 
    dataframe객체.agg( { '열이름' : [func1, ...] , '열이름2' : ..... } )
    dataframe객체.agg( { '열이름' : [ ( 결과 반환 열 이름, func1 ) , (   ,   ) ] , '열이름2' : [   .....   }  )
#### 여러 함수를 한번에 적용

In [42]:
train = pd.read_csv('C://python/DATA_SET/Taitanic/train.csv')
train


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [43]:
g_train = train[['Fare','Age']].groupby(train['Embarked'], axis = 0)

    1. dataframe객체.agg( [ func1, func2, ....] )

In [106]:
g_train.agg(['mean','std',peak_to_peak]).head(6)

Fare                                Age                        
               mean        std peak_to_peak       mean        std peak_to_peak
Embarked                                                                      
C         59.954144  83.912994     508.3167  30.814769  15.434860        70.58
Q         13.276030  14.188047      83.2500  28.089286  16.915396        68.50
S         27.079812  35.887993     263.0000  29.445397  14.143192        79.33

    2. dataframe객체.agg( [ (결과 반환 열 이름 , func1) , (  ,  ) ....] ) 

( 원하는 칼럼 이름, 적용하는 함수 ) 라는 튜플을 agg( ) 괄호 안에 리스트 형태의 인자로 넣어준다.

In [44]:
g_train.agg([('평균','mean'),('최댓값','max'),('최솟값','min')]).head(10)

# 전체 각각 열에 대해
# mean, max, min 함수를 수행하고
# 각 결과물을 '평균' , '최댓값' , '최솟값' 이라는 열을 생성하여
# dataframe 형태로 반환한다.

Fare                          Age            
                 평균       최댓값     최솟값         평균   최댓값   최솟값
Embarked                                                    
C         59.954144  512.3292  4.0125  30.814769  71.0  0.42
Q         13.276030   90.0000  6.7500  28.089286  70.5  2.00
S         27.079812  263.0000  0.0000  29.445397  80.0  0.67

In [45]:
func = [('평균','mean'),('분산','std'),('메디안','median')]

g_train_res = g_train.agg(func) # 데이터 프레임 형태이므로 다룰 수 있다.
g_train_res.head() 

Fare                          Age                 
                 평균         분산    메디안         평균         분산   메디안
Embarked                                                         
C         59.954144  83.912994  29.70  30.814769  15.434860  29.0
Q         13.276030  14.188047   7.75  28.089286  16.915396  27.0
S         27.079812  35.887993  13.00  29.445397  14.143192  28.0

In [109]:
g_train_res['Age'].head()

,평균,분산,메디안
Embarked,,,
C,30.814769,15.434860,29.0
Q,28.089286,16.915396,27.0
S,29.445397,14.143192,28.0


3. dataframe객체.agg( { '열이름' : [func1, ...] , '열이름2' : ..... } )

딕셔너리의 key : value 
- key : 적용할 열
- value : 적용할 함수들

In [46]:
g_train.agg({'Fare' : ['mean','std'], 'Age' : ['max','min']})

# Passengerld 열에는 mean, std 함수를 적용하고
# Age 열에는 max, min 함수를 적용한다.

Fare              Age      
               mean        std   max   min
Embarked                                  
C         59.954144  83.912994  71.0  0.42
Q         13.276030  14.188047  70.5  2.00
S         27.079812  35.887993  80.0  0.67

In [112]:
g_train.agg({'Fare' : [('평균값','mean'),('분산','std')], 'Age' : [('최대','max'),('최소','min')]}).head()

# 각각 컬럼에 대해 다른 함수를 적용하기 - 딕셔너리 이용
# 함수 리스트를 주면 함수 이름으로 컬럼이 생성
# 적용된 함수로 생성된 컬럼 이름을 다른 것으로 설정하고 싶은 경우 ( 이름, 함수 ) 형식으로 튜플 이용


Fare              Age      
                평균값         분산    최대    최소
Embarked                                  
C         59.954144  83.912994  71.0  0.42
Q         13.276030  14.188047  70.5  2.00
S         27.079812  35.887993  80.0  0.67

## 9.2.2 색인되지 않은  집계된 데이터 반환하기
default : 그룹화 후 결과에서 그룹 기준 key 는 인덱스로 간다.

groupby 메서드로 묶은 후 묶어진 인덱스를 사용하지 않고 새로운 인덱스(0,1,2...) 를 사용하고 싶을 때
    
    dataframe객체.groupby(     ,    as_index = False   )사용

In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [51]:
train.groupby(['Pclass','Cabin']).mean().head() # as_index 옵션을 설정하지 않은 경우

PassengerId  Survived   Age  SibSp  Parch     Fare
Pclass Cabin                                                    
1      A10          584.0       0.0  36.0    0.0    0.0  40.1250
       A14          476.0       0.0   NaN    0.0    0.0  52.0000
       A16          557.0       1.0  48.0    1.0    0.0  39.6000
       A19          285.0       0.0   NaN    0.0    0.0  26.0000
       A20          600.0       1.0  49.0    1.0    0.0  56.9292

In [114]:
train.groupby(['Pclass','Cabin'], as_index = False).mean().head()
# as_index = False 옵션을 주어 Pclass와 Cabin 이 좌측 인덱스로 가지 않는다.

,Pclass,Cabin,PassengerId,Survived,Age,SibSp,Parch,Fare
0,1,A10,584.0,0.0,36.0,0.0,0.0,40.1250
1,1,A14,476.0,0.0,NaN,0.0,0.0,52.0000
2,1,A16,557.0,1.0,48.0,1.0,0.0,39.6000
3,1,A19,285.0,0.0,NaN,0.0,0.0,26.0000
4,1,A20,600.0,1.0,49.0,1.0,0.0,56.9292


## 9.3 그룹별 연산과 변형

다양한 그룹 연산 수행
    
    transform
    apply

In [54]:
df

,key1,key2,data1,data2
0,a,one,1.485902,1.300076
1,a,two,0.520862,-0.482945
2,b,one,0.178584,0.277137
3,b,two,-1.349532,-0.085610
4,a,one,-0.941951,0.368759


In [48]:
k1_mean = df.groupby('key1').mean().add_prefix('평균_')
k1_mean

,평균_data1,평균_data2
key1,,
a,-0.083769,-1.187640
b,-0.383423,-0.047439


In [49]:
pd.merge(df, k1_mean , left_on = 'key1', right_index = True ) 

# left_on = 'key1' : 좌측에 있는 데이터(df) 에서는 'key1' 을
# right_index = True : 우측에 있는 데이터(k1_mean) 에서는 '인덱스 값'을
# key 로 삼아 둘을 병합한다.

,data1,data2,key1,key2,평균_data1,평균_data2
0,-0.361864,0.112269,a,one,-0.083769,-1.187640
1,-0.437168,-2.368774,a,two,-0.083769,-1.187640
4,0.547725,-1.306415,a,one,-0.083769,-1.187640
2,-0.722249,-1.047286,b,one,-0.383423,-0.047439
3,-0.044597,0.952409,b,two,-0.383423,-0.047439


### transform 을 이용한 좋은 방법

In [57]:
people

,a,b,c,d,e
joe,0,19.0,1.0,1,7
steve,7,15.0,16.0,7,9
wes,19,NaN,NaN,17,18
jim,8,19.0,10.0,10,5
travis,11,6.0,10.0,8,3


In [58]:
key = ['one','two','one','two','one']

In [59]:
# groupby 의 기본 축은 axis = 0임을 잊지 말자
people.groupby(key).mean()

,a,b,c,d,e
one,10.0,12.5,5.5,8.666667,9.333333
two,7.5,17.0,13.0,8.500000,7.000000


In [62]:
# transform 용도는 다시 공부해야겠다.
people.transform([np.exp, np.sqrt])

a                       b                       c  \
                 exp      sqrt           exp      sqrt           exp   
joe     1.000000e+00  0.000000  1.784823e+08  4.358899  2.718282e+00   
steve   1.096633e+03  2.645751  3.269017e+06  3.872983  8.886111e+06   
wes     1.784823e+08  4.358899           NaN       NaN           NaN   
jim     2.980958e+03  2.828427  1.784823e+08  4.358899  2.202647e+04   
travis  5.987414e+04  3.316625  4.034288e+02  2.449490  2.202647e+04   

                             d                       e            
            sqrt           exp      sqrt           exp      sqrt  
joe     1.000000  2.718282e+00  1.000000  1.096633e+03  2.645751  
steve   4.000000  1.096633e+03  2.645751  8.103084e+03  3.000000  
wes          NaN  2.415495e+07  4.123106  6.565997e+07  4.242641  
jim     3.162278  2.202647e+04  3.162278  1.484132e+02  2.236068  
travis  3.162278  2.980958e+03  2.828427  2.008554e+01  1.732051

### 9.3.1 apply : 분리- 적용- 병합
#### aggregate, transform 함수 : 엄격한 요구사항을 갖는 '특수한 목적의 함수'
#### 스칼라 값을 생성 / 같은 크기를 갖는 변형된 배열 생성
#### apply : 일반적인 목적을 갖는 함수
#### 객체를 여러조각으로 나누고 각 조각에 함수를 적용 합치기
#### apply 를 독창적으로 사용하는 방법
#### apply(함수, 함수의 인자들 설정)
#### 예를 들면 func(a,b,c) :라는 함수가 있다면
#### apply( func , a =  , b =  , c =  ) 로 함수의 옵션을 넘겨주어 apply 메서드 사용 가능

In [53]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
train.groupby(['Embarked','Cabin']).mean().head()

PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare
Embarked Cabin                                                            
C        A10          584.0       0.0     1.0  36.0    0.0    0.0  40.1250
         A16          557.0       1.0     1.0  48.0    1.0    0.0  39.6000
         A20          600.0       1.0     1.0  49.0    1.0    0.0  56.9292
         A26          648.0       1.0     1.0  56.0    0.0    0.0  35.5000
         A31          210.0       1.0     1.0  40.0    0.0    0.0  31.0000

In [55]:
train.groupby(['Embarked','Cabin'], group_keys = False).mean().head()

PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare
Embarked Cabin                                                            
C        A10          584.0       0.0     1.0  36.0    0.0    0.0  40.1250
         A16          557.0       1.0     1.0  48.0    1.0    0.0  39.6000
         A20          600.0       1.0     1.0  49.0    1.0    0.0  56.9292
         A26          648.0       1.0     1.0  56.0    0.0    0.0  35.5000
         A31          210.0       1.0     1.0  40.0    0.0    0.0  31.0000

### 9.3.2 변위치 분석과 버킷 분석
#### groupby 와 cut의 조합
#### cut으로 나눈 구간을 groupby의 인자로 넣어준다.

In [87]:
frame = pd.DataFrame({'data1' : np.random.randn(1000), 
                      'data2' : np.random.randn(1000)})

In [88]:
frame.head()

,data1,data2
0,-0.900453,-0.405782
1,0.691061,-1.591658
2,-0.410675,1.427592
3,-0.529418,-0.538215
4,-0.768931,0.827746


In [ ]:
pd.cut()

In [91]:
factor = pd.cut(frame['data1'], 4)

In [100]:
factor[:10]

0    (-1.827, -0.268]
1     (-0.268, 1.292]
2    (-1.827, -0.268]
3    (-1.827, -0.268]
4    (-1.827, -0.268]
5    (-1.827, -0.268]
6     (-0.268, 1.292]
7    (-1.827, -0.268]
8    (-1.827, -0.268]
9    (-1.827, -0.268]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.393, -1.827] < (-1.827, -0.268] < (-0.268, 1.292] < (1.292, 2.851]]

In [103]:
def get_stats(group):
    return { '최소' : group.min(), '최대' : group.max(), '개수' : group.count(), '평균' : group.mean()} 

grouped = frame['data2'].groupby(factor)

grouped.apply(get_stats)
# 사용자가 만든 함수를 넣어 각 그룹에 적용가능
# 이와 같은 형태로 나온다

data1               
(-3.393, -1.827]  개수     30.000000
                  최대      2.134487
                  최소     -2.199158
                  평균     -0.088780
(-1.827, -0.268]  개수    354.000000
                  최대      3.270799
                  최소     -3.192078
                  평균      0.007706
(-0.268, 1.292]   개수    523.000000
                  최대      3.083623
                  최소     -2.968835
                  평균      0.001894
(1.292, 2.851]    개수     93.000000
                  최대      2.628990
                  최소     -2.314323
                  평균     -0.213025
Name: data2, dtype: float64

In [62]:
grouped.apply(get_stats).unstack() # unstack을 이용하여 DataFrameㅎㅇ태로 만들어준다.

,개수,최대,최소,평균
data1,,,,
"(-3.145, -1.574]",62.0,1.974692,-2.614653,0.057311
"(-1.574, -0.00889]",449.0,3.635455,-3.254016,-0.032702
"(-0.00889, 1.556]",441.0,3.209489,-2.877410,0.106037
"(1.556, 3.121]",48.0,2.677619,-2.241377,0.042285


In [63]:
# 변위치의 숫자를 반환한다. 데이터의 qunatile 을 계산해서 각 그룹의 크기가 동일하도록 구간을 정해서 나누어준다.
# labels = False 옵션
grouping = pd.qcut(frame['data1'], 10 , labels = False)
grouping.head()


0    3
1    6
2    7
3    3
4    7
Name: data1, dtype: int64

In [64]:
pd.qcut(frame['data1'],10  ).head()

0    (-0.507, -0.241]
1      (0.229, 0.519]
2      (0.519, 0.817]
3    (-0.507, -0.241]
4      (0.519, 0.817]
Name: data1, dtype: category
Categories (10, interval[float64]): [(-3.139, -1.27] < (-1.27, -0.854] < (-0.854, -0.507] < (-0.507, -0.241] ... (0.229, 0.519] < (0.519, 0.817] < (0.817, 1.231] < (1.231, 3.121]]

In [65]:
grouped2 = frame['data2'].groupby(grouping) # grouping 의 구간 객체를 통해 frame 의 data2 열을 그룹화한다.
grouped2.apply(get_stats).unstack() # 그룹화한 데이터에 get_stats 함수를 적용하고 unstack을 적용하여 dataframe 을 만든다.

,개수,최대,최소,평균
data1,,,,
0,100.0,1.983414,-3.254016,0.038213
1,100.0,2.286327,-2.417778,0.005464
2,100.0,3.635455,-2.035032,-0.007117
3,100.0,1.831934,-2.732274,-0.162746
4,100.0,3.045529,-2.591611,-0.009100
5,100.0,2.576473,-1.957091,0.319417
6,100.0,2.593933,-2.166437,-0.054755
7,100.0,3.209489,-2.109011,0.027656
8,100.0,2.614196,-2.877410,0.168720


### 9.3.3 예제: 그룹에 국한된 값으로 누락된 값 채우기
#### dropna(how = 'all' or 'any') : Nan 값을 포함한 열 지우기(모두 Nan 인 경우, 하나라도 Nan인 경우)
#### fillna : 고정 값이나, 해당 열에서 추출된 값으로 채우기


#### 그룹별로 채우고 싶은 값이 다른 경우
#### 데이터를 그룹으로 나눈다 -> apply 를 이용하여 각 그룹에 fillna 적용


In [104]:
train['Pclass'].fillna(method = 'ffill').head()

0    3
1    1
2    3
3    1
4    3
Name: Pclass, dtype: int64

In [105]:
True in train['Pclass'].isnull().values # nan 모두 채워짐

False

In [112]:
Pcls_train = train.groupby(train['Pclass'])

In [113]:
group_fillmean = lambda g : g.fillna(g.mean()) # 각 그룹별로 그룹의 평균값을 채우는 사용자 지정 메서드

In [114]:
no_nan_train = Pcls_train.apply(group_fillmean) 
# Pclass를 기준으로 그룹화 한 후(Pcls_train) 
# 각 열에서 각 그룹의 평균 값으로 Nan을 채운다.(apply(group_fillmean))

In [71]:
istherenull = []
for i in no_nan_train.columns:
    a  = (True in no_nan_train[i].isnull().values)
    istherenull.append(a)

    

In [72]:
istherenull # Cabin과 Embarked 를 제외한 모든 열이 Pclass를 기준으로 한 그룹의 평균값으로 nan 값이 채워졌다.

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True]

### 9.3.4 예제 : 랜덤 표본과 순열
#### 효과적으로 표본 추출하기 : np.random.permutataion(N) : 처음 K 원소 선택
#### N : 전체 데이터셋 크기 / K : 원하는 표본 크기

In [116]:
arr = [i for i in range(100)]

In [117]:
arr

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [120]:
np.random.permutation(arr)[:int(len(arr) * 0.6)]

array([35, 38, 68, 75, 25, 99, 16, 21, 58, 12, 31, 67, 44, 28, 90, 18, 92,
       97, 37, 42, 54, 64,  4, 51, 69, 79, 93, 78, 70, 43, 47, 57, 49, 11,
        2, 85, 96,  6, 19, 53, 14, 20, 26, 63, 48, 83, 56, 27,  0, 86,  1,
       61, 39, 23, 50, 77, 76, 84, 24, 66])

In [121]:
from sklearn.model_selection import train_test_split

In [122]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [73]:
# 하트, 스페이드, 클로버, 다이아몬드
suits = ['H','S','C','D'] # 각 군집
card_val = (list(range(1,11)) + [10] *3 ) *4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q'] # 각 군집의 카드 이름( A,2,3,4,.....,J,K,Q)

cards = []
for suit in ['H','S','C','D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

In [74]:
deck[:20]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
dtype: int64

In [75]:
def draw(cardsdeck, n = 5):
    return cardsdeck.take(np.random.permutation(len(deck))[:n])
# 카드를 무작위 추출하는 함수를 만든다.

In [76]:
draw(deck)

5C    5
4H    4
4S    4
2S    2
3S    3
dtype: int64

In [77]:
# 각 무늬별로 2장씩 뽑고 싶은 경우
# 각 무늬를 기준으로 그룹화 한 후 , draw 함수를 apply에 적용
get_suit = lambda card : card[-1] # 마지막 글자가 무늬를 나타내므로

deck.groupby(get_suit).apply(draw , n = 2)
# 마지막 글자(무늬)를 반환하는 함수를 적용하여 무늬별로 그룹화한 후 
# 무작위로 뽑는 draw 함수를 적용하여 카드를 뽑는다

# 즉 그룹별( C ,D ,H, S) 로 각각 draw 함수가 적용되어
# 각 그룹별로 카드가 2개씩 뽑힌다.

# 여기서 apply( 함수 이름 , 함수 옵션 설정 ) 을 엿볼 수 있다.
# 원래 draw 함수는 default 값이 n = 5 로 되어 있는데
# apply 에 옵션으로 함수 옵션을 변경하여 실행시킬 수 있다.

IndexError: index 43 is out of bounds for axis 0 with size 13

### 9.3.5 예제 : 그룹 가중 평균과 상관관계


In [ ]:
wdf = pd.DataFrame({'categories' : list('aaaabbbb'),
                    'data' : np.arange(8,0,-1),
                    'weights' : np.arange(8)})
wdf

In [ ]:
# categories를 기준으로 그룹을 나눈다.
grouped_w = wdf.groupby('categories')

In [ ]:
# weight 열에 따라 가중평균을 구하는 함수를 만든다.
# 각 데이터에 매칭되는 가중치가 들어있는 열이나 Seires 가 있어야한다.

get_wavg = lambda g : np.average(g['data'], weights = g['weights'])

# np.average(  , weights ) 옵션을 통해 가중치를 준다.

In [ ]:
grouped_w.apply(get_wavg)

In [ ]:
train.head()

In [ ]:
train['Age'].pct_change()

### 9.3.6 예제 : 그룹 상의 선형 회귀
####  통계 분석 : statsmodels 라이브러리 - regress 함수 작성

##  9.4 피벗 테이블과 교차알람표 - 잘 모르겠음...
#### 피벗 테이블 : 데이터 요약화 도구
#### key로 데이터를 수집하여 어떤 key는 행에, 어떤 key는 열에 나열한다.
#### groupby와 사용하여 계층적 색인에 사용하면 : 재형성 연산 가능
#### DF.pivot_table(rows = , columns = ) 메서드

In [ ]:
#train.head()

In [ ]:
#train.pivot_table('Sex')

In [ ]:
#train.groupby("Sex").pivot_table()

### 9.4.1 교차형 일람표
### 범주형 데이터들의 교차식 빈도수(개수) 를 셀 때 사용
#### 그룹 빈도를 계산하기 위한 피벗테이플의 특수한 경우

In [131]:
data = pd.DataFrame({'Sample': [i for i in range(1,11)],
              'Gender': ['F','M','F','M','M','M','F','F','M','F'], 
              'Handedness' : ['R','L','R','R','L','R','R','L','R','R'],
              'Glass' : ['Y','N','N','Y'] * 2 + ['Y','Y']})
data

,Sample,Gender,Handedness,Glass
0,1,F,R,Y
1,2,M,L,N
2,3,F,R,N
3,4,M,R,Y
4,5,M,L,Y
5,6,M,R,N
6,7,F,R,N
7,8,F,L,Y
8,9,M,R,Y
9,10,F,R,Y


In [135]:
pd.crosstab(data['Gender'], data['Handedness'], margins = True)a


Handedness,L,R,All
Gender,,,
F,1,4,5
M,2,3,5
All,3,7,10


In [ ]:
pd.crosstab([data['Gender'], data['Handedness']], data['Glass'], margins = True) # 계층색인도 가능ㅇ